
#**Model Attempt #4: FINAL MODEL**

Improvments to be made:

ResNet50- deeper CNN, ImageNet training via Pytorch

Weights to handle data imbalance (CrossEntropyLoss)

Data transformation

Learning rate reducer

Early stopping

Model

In [24]:
#load in libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from collections import Counter
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import random
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
import os
from imageio import imread

#stop warnings
import warnings
warnings.filterwarnings('ignore')

#from google.colab import drive
#drive.mount('/content/drive')
#!unzip -q /content/drive/MyDrive/dataset.zip -d /content/dataset
path = '/content/dataset/dataset'

In [ ]:
#initialize
width = []
height = []
channel_color = []
weather_type = []

#set up path
for folder in os.listdir(path):
    subfolder_path = os.path.join(path, folder)
    for image_file in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_file)
        image = imread(image_path)
        if len(image.shape) < 3:
            image = image.reshape(image.shape + (1,))
        h, w, c = image.shape
        #update lists
        width.append(w)
        height.append(h)
        channel_color.append(c)
        weather_type.append(folder)

#make lists a df
df = pd.DataFrame({'width': width, 'height': height, 'channels': channel_color, 'weather': weather_type})
df.head()

,width,height,channels,weather
0,217,275,3,rime
1,400,266,3,rime
2,400,300,3,rime
3,400,266,3,rime
4,400,261,3,rime


Model

In [ ]:
#make sure gpu is working
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

In [ ]:
#import misc libraries; could be needed
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from collections import Counter
import copy

#transform
transformation_recipe1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transformation_recipe2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

#get dataset and transform
weather_data = datasets.ImageFolder(path, transform = transformation_recipe1)

#weighted loss
labels = []
for i in weather_data:
    label = i[1]
    labels.append(label)
class_counts = Counter(labels)
total_samples = sum(class_counts.values())
class_weights = []
for i in range(len(weather_data.classes)):
    weight_amt = total_samples / class_counts[i]
    class_weights.append(weight_amt)

#split
train_size = int(0.8 * len(weather_data))
test_size = len(weather_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(weather_data, [train_size, test_size])

#transform test
test_dataset.dataset.transform = transformation_recipe2
load_train = DataLoader(train_dataset, batch_size = 32, shuffle = True)
load_test = DataLoader(test_dataset, batch_size = 32)

#model
resnet_model = models.resnet50(weights = models.ResNet50_Weights.IMAGENET1K_V1)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, len(weather_data.classes))
resnet_model = resnet_model.to(dev)

#optimize
crit = nn.CrossEntropyLoss(weight = torch.FloatTensor(class_weights).to(dev))
optimizer = optim.Adam(resnet_model.parameters(), lr = 0.0001)
lr_reducer = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.5, patience = 2)

#train
epochs = 15
best_test_acc = 0
p_counter = 0
for epoch in range(epochs):
    resnet_model.train()
    for images, labels in load_train:
        images, labels = images.to(dev), labels.to(dev)
        optimizer.zero_grad()
        crit(resnet_model(images), labels).backward()
        optimizer.step()
    #test
    resnet_model.eval()
    correct, total, test_loss = 0, 0, 0
    with torch.no_grad():
        for images, labels in load_test:
            images, labels = images.to(dev), labels.to(dev)
            test_loss = test_loss + crit(resnet_model(images), labels).item()
            predicted = torch.argmax(resnet_model(images), dim = 1)
            total = total + labels.size(0)
            correct = correct + (predicted == labels).sum().item()
    avg_test_loss = test_loss / len(load_test)
    test_acc = 100 * correct / total
    print(f"Epoch #{epoch+1}: Accuracy = {test_acc:.3f}%")
    lr_reducer.step(avg_test_loss)
    #stop early if need
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        p_counter = 0
    else:
        p_counter = p_counter + 1
        if p_counter >= 5:
            break

#get highest accuracy
print(f"Highest Accuracy Rate: {best_test_acc:.3f}%")


Epoch #1: Accuracy = 87.473%
Epoch #2: Accuracy = 89.585%
Epoch #3: Accuracy = 89.366%
Epoch #4: Accuracy = 87.983%
Epoch #5: Accuracy = 87.546%
Epoch #6: Accuracy = 92.280%
Epoch #7: Accuracy = 91.916%
Epoch #8: Accuracy = 92.498%
Epoch #9: Accuracy = 90.605%
Epoch #10: Accuracy = 91.333%
Epoch #11: Accuracy = 91.770%
Epoch #12: Accuracy = 91.770%
Epoch #13: Accuracy = 92.061%
Highest Accuracy Rate: 92.498%


Graphs